# Homework Solutions: Docker, PostgreSQL, and Data Analysis

## Question 1: Docker First Run
**Task**: Check the `pip` version in the `python:3.12.8` Docker image.

### Solution
```bash
# Run the Docker container with bash entrypoint
docker run -it --entrypoint bash python:3.12.8

# Inside the container, check pip version
pip --version
```
**Output**: `pip 24.3.1`

**Answer**: `24.3.1`

------------------------------

## Question 2: Docker Networking
**Task**: Determine the hostname and port for pgAdmin to connect to PostgreSQL.

docker-compose.yaml
```yaml
# Provided docker-compose.yaml
services:
  db:
    container_name: postgres
    image: postgres:17-alpine
    environment: [...]
    ports: ["5433:5432"]
  pgadmin:
    # ...
```
**Explanation**
- **Hostname**: `db` (Docker Compose service name)

- **Port**: `5432` (internal PostgreSQL port)

- Port mapping `5433:5432` is for the host machine, not inter-container communication.

**Answer**: `db:5432`

------------------------------


## Question 3: Trip Segmentation Count

**Task**: Categorize trips by distance from Oct 1 to Nov 1, 2019.

**SQL Query**
```sql
WITH segmented_trips AS (
  SELECT
    CASE
      WHEN trip_distance <= 1 THEN '0-1'
      WHEN trip_distance > 1 AND trip_distance <= 3 THEN '1-3'
      WHEN trip_distance > 3 AND trip_distance <= 7 THEN '3-7'
      WHEN trip_distance > 7 AND trip_distance <= 10 THEN '7-10'
      ELSE '>10'
    END AS segment
  FROM green_taxi_trips
  WHERE lpep_pickup_datetime >= '2019-10-01' 
    AND lpep_pickup_datetime < '2019-11-01'
)
SELECT 
  COUNT(*) FILTER (WHERE segment = '0-1') AS up_to_1,
  COUNT(*) FILTER (WHERE segment = '1-3') AS between_1_3,
  COUNT(*) FILTER (WHERE segment = '3-7') AS between_3_7,
  COUNT(*) FILTER (WHERE segment = '7-10') AS between_7_10,
  COUNT(*) FILTER (WHERE segment = '>10') AS over_10
FROM segmented_trips;
```

**Answer**: `104,793; 202,661; 109,603; 27,678; 35,189` (Option 4)

-------------------------



## Question 4: Longest Trip per Day

**Task**: Find the day with the longest trip distance.

**SQL Query**

```sql
SELECT DATE(lpep_pickup_datetime) AS pickup_day, MAX(trip_distance)
FROM green_taxi_trips
GROUP BY pickup_day
ORDER BY MAX(trip_distance) DESC
LIMIT 1;
```

**Answer**: `2019-10-31` (Option D)

------------------------

## Question 5: Top 3 Pickup Zones (2019-10-18)

**Task**: Zones with >13,000 total_amount on 2019-10-18.

**SQL Query**

```sql
SELECT 
  z."Zone" AS pickup_zone,
  SUM(t.total_amount) AS total
FROM green_taxi_trips t
JOIN taxi_zone_lookup z 
  ON t."PULocationID" = z."LocationID"
WHERE DATE(lpep_pickup_datetime) = '2019-10-18'
GROUP BY z."Zone"
HAVING SUM(t.total_amount) > 13000
ORDER BY total DESC
LIMIT 3;
```

**Answer**: `East Harlem North, East Harlem South, Morningside Heights` (Option A)

------------------------


## Question 6: Largest Tip from East Harlem North

**Task**: Find drop-off zone with the largest tip.

**SQL Query**

```sql
SELECT 
  dz."Zone" AS dropoff_zone,
  MAX(t.tip_amount) AS largest_tip
FROM green_taxi_trips t
JOIN taxi_zone_lookup pz 
  ON t."PULocationID" = pz."LocationID"
JOIN taxi_zone_lookup dz 
  ON t."DOLocationID" = dz."LocationID"
WHERE pz."Zone" = 'East Harlem North'
  AND DATE(lpep_pickup_datetime) BETWEEN '2019-10-01' AND '2019-10-31'
GROUP BY dz."Zone"
ORDER BY largest_tip DESC
LIMIT 1;
```

**Answer**: `JFK Airport` (Option B)

--------------------------